In [1]:
class Node:
    """ The Node class is used in the search algorithms.
    It can be trivially extended to include costs
    """
    def __init__(self, state, parent=None, action=None, cost=0):
        """ The constructor needs the current state and action, and a pointer to a
        parent Node.
        For the 'root' node, only the start state.
        """
        self.state = state
        self.parent = parent
        self.action = action
        self.cost = cost
        
    def __str__(self):
        """ Returns a string with the state and action of this node"""
        return f"State: {self.state} Action: {self.action}\n"


In [2]:
class Environment(object):
    """ This class is model specific (not universal at all!)"""
    def __init__(self, problem):
        self.start = problem[0] # In our case: [3,3,0,0,-1]
        self.goal = problem[1] # In our case: [0,0,3,3,1]
        

In [27]:
class SearchAgent:
    """ Generic class for search algorithms"""
    def __init__(self,environment):
        """ The constructor requires an environment class that, at least contains
        the start and goal states.
        
        If additional information is required, this class can be inherited but you
        should include:
        super().__init__()
        """
        self.frontier = [Node(state=environment.start)] # We play with nodes, not states.
        self.explored = set() # I use a 'set' to avoid repetitions (and order doesn't matter)
        self.goal_state = environment.goal
    
    def print_frontier(self): 
        """ Print the nodes in the frontier"""
        print("Frontier:")
        print("---------")
        for node in self.frontier:
            print(node)
        print("---------")
            
    def print_explored(self):
        """ Print the nodes in the explored set"""
        print("Explored:")
        print("---------")
        for node in self.explored:
            print(node)
        print("---------")
            
    def is_frontier_empty(self):
        """ A boolean. True if the frontier is empty (so there is no solution)"""
        return len(self.frontier) == 0 # No solution

    def actions(self,node):
        """ Needs to be extended in the child daughter classes of this class.
        This function is the main function codifiying the problem and it has 
        to be specific to the subject at hand.
        """
        pass
    
    def result(self,node):
        """ Needs to be extended in the child daughter classes of this class.
        This function is the main function codifiying the problem and it has 
        to be specific to the subject at hand.
        """
        pass
    
    def remove_node(self):
        """ Needs to be extended in the child daughter classes of this class.
        Here is where we decide if we use a stack, a queue, ...
        """
        pass
    
    def is_goal(self,node):
        """ This function plays two roles:
        1. Check if we have currently reached the goal state.
        2. If so, reconstruct the tree backwards to show the obtained solution.
        """
        if node.state == self.goal_state:
            self.solution = []
            while node.parent is not None:
                self.solution.append(node)
                print(f"{node.state}({node.action})")
                node = node.parent
            raise Exception("Solution found")

    def add_to_frontier(self,node):
        """ Append node to the frontier. """
        self.frontier.append(node)
            
    def add_to_explored(self,node):
        """Add node to explored set (this is a pythonic "set" that's why I use add instead of append."""
        self.explored.add(node) 

    def is_state_in_frontier(self,node):
        """Returns a boolean (True) if the state is in the current frontier"""
        return any(frontier_node.state == node.state for frontier_node in self.frontier)
  
    def is_state_in_explored(self,node):
        """Returns a boolean (True) if the state is in the explored set"""
        return any(explored_node.state == node.state for explored_node in self.explored)
    
    def expand_node(self, node):
        """Here is the main function of the code. We decide RESULTs taking the states and 
        actions into consideration.
        
        I've tried to make this code as versatile as I can. This part should be the same
        for any search problem you program. Cool, uh!
        """
        for child in self.result(node): 
            # Check if the node is not frontier or explored
            if not self.is_state_in_frontier(child) and not self.is_state_in_explored(child):
                self.add_to_frontier(child) # add all the "compatible" children
        
    def solve(self):
        """The main loop. Here is where we implement the "Repeat" part of the algorithm. 
        It follows almost verbatim the description given in the lecture"""
        while not self.is_frontier_empty():
            #self.print_frontier()
            #self.print_explored()
            try:
                node = self.remove_node()
            except:
                print("Solution not found!!")
                
            try:
                self.is_goal(node)
            except:
                return self.solution
            self.add_to_explored(node)
            self.expand_node(node)

In [29]:
class AgentDepthFirst(SearchAgent):
    def remove_node(self):
        return self.frontier.pop() # Remove the last element (this is a stack!!!)
    

class RiverSearch(AgentDepthFirst):
    def actions(self,node):
        name = {-1:"left",1:"right"}
        ml, wl, mr, wr, side = node.state
        
        possible_actions = [
            (f"Move 1 man from {name[side]} side",(ml+side,wl,mr-side,wr,-side)),
            (f"Move 2 men from {name[side]} side",(ml+2*side,wl,mr-2*side,wr,-side)),
            (f"Move 1 man and 1 woman from {name[side]} side",(ml+side,wl+side,mr-side,wr-side,-side)),
            (f"Move 1 woman from {name[side]} side",(ml,wl+side,mr,wr-side,-side)),
            (f"Move 2 women from {name[side]} side",(ml,wl+2*side,mr,wr-2*side,-side))

        ]
        #print(f"Potential: from {node}\n{possible_actions}")
        return possible_actions
        
    def result(self,node):
        children = []
        for action, (ml,wl,mr,wr,s) in self.actions(node):
            if (ml>=wl or ml==0) and (mr>=wr or mr==0) and ml<=3 and mr<=3 and wl<=3 and wr<=3:
                #print(f"Action passed:{action},{(ml,wl,mr,wr,s)}")
                children.append(Node(state=(ml,wl,mr,wr,s),parent=node,action=action))
                
        return children
        
        

In [31]:
solver = RiverSearch(Environment([(3,3,0,0,-1),(0,0,3,3,1)]))
solver.solve();

(0, 0, 3, 3, 1)(Move 2 women from left side)
(0, 2, 3, 1, -1)(Move 1 woman from right side)
(0, 1, 3, 2, 1)(Move 2 women from left side)
(0, 3, 3, 0, -1)(Move 1 woman from right side)
(0, 2, 3, 1, 1)(Move 2 men from left side)
(2, 2, 1, 1, -1)(Move 1 man and 1 woman from right side)
(1, 1, 2, 2, 1)(Move 2 men from left side)
(3, 1, 0, 2, -1)(Move 1 woman from right side)
(3, 0, 0, 3, 1)(Move 2 women from left side)
(3, 2, 0, 1, -1)(Move 1 woman from right side)
(3, 1, 0, 2, 1)(Move 2 women from left side)


In [32]:
class RiverSearchDFS(RiverSearch):
    def remove_node(self):
        return self.frontier.pop(0) # Remove the first element (this is a queue!)

In [34]:
solver = RiverSearchDFS(Environment([(3,3,0,0,-1),(0,0,3,3,1)]))
solver.solve();

(0, 0, 3, 3, 1)(Move 1 man and 1 woman from left side)
(1, 1, 2, 2, -1)(Move 1 man from right side)
(0, 1, 3, 2, 1)(Move 2 women from left side)
(0, 3, 3, 0, -1)(Move 1 woman from right side)
(0, 2, 3, 1, 1)(Move 2 men from left side)
(2, 2, 1, 1, -1)(Move 1 man and 1 woman from right side)
(1, 1, 2, 2, 1)(Move 2 men from left side)
(3, 1, 0, 2, -1)(Move 1 woman from right side)
(3, 0, 0, 3, 1)(Move 2 women from left side)
(3, 2, 0, 1, -1)(Move 1 man from right side)
(2, 2, 1, 1, 1)(Move 1 man and 1 woman from left side)


Esta es la solución que podéis encontrar en la Wikipedia: [Cannibals and Missionaires](https://en.wikipedia.org/wiki/Missionaries_and_cannibals_problem)